# Vamos aprender a trabalhar com PDF usando o Python

- Regra geral: PDF foi feito justamente para bloquear muita coisa, então não é fácil "brincar" com um pdf
- Mesmo assim, Python tem várias bibliotecas que vão nos ajudar, vamos focar em 2:
    - PyPDF2
    - Tabula
- Ler e extrair informações de um PDF a gente consegue fazer.
- Escrever e Editar, aí já é outra história

### Para os nossos exemplos, vamos avaliar o Release de Resultados do 3º e 4º Trimestre de 2020 da Magazine Luiza

#### 1º Objetivo: Queremos conseguir separar apenas o DRE do Release de Resultados (Página 14) para enviar para a Diretoria, como fazemos?
    - Separar as páginas de um pdf

In [66]:
import PyPDF2 as pyf
from pathlib import Path


nome_arquivo = "MGLU_ER_3T20_POR.pdf"
arquivo_pdf = pyf.PdfReader(nome_arquivo)

i = 1
for pagina in arquivo_pdf.pages:
    arquivo_novo = pyf.PdfWriter()
    arquivo_novo.add_page(pagina)
    #salvar o arquivo
    with Path(f'paginas/Arquivo{i}.pdf').open(mode='wb') as arquivo_final:
        arquivo_novo.write(arquivo_final)
        i += 1

#### 2º Objetivo: Com o Release de Resultados já separado página por página, queremos incluir apenas as Páginas de Destaque (Página 1), DRE (Página 14) e Balanço (Página 16).
    - Juntar vários pdfs em 1

In [67]:
paginas = [1, 14, 16]

i = 1
#criar um novo pdf
novo_pdf = pyf.PdfWriter()
for num_pagina in paginas:
    nome_arquivo = f"paginas/Arquivo{num_pagina}.pdf"
    arquivo_pdf = pyf.PdfReader(nome_arquivo)
    pagina = arquivo_pdf.pages[0]
    novo_pdf.add_page(pagina)
    # pegar o arquivo num_pagina
    #adicionar ele no novo pdf

#salvar o novo pdf
with Path(f'consolidado.pdf').open(mode='wb') as arquivo_final:
    novo_pdf.write(arquivo_final)
     


### Extra: Para adicionar todas as páginas de 2 pdfs

In [68]:
pdf_mesclado = pyf.PdfMerger()

arquivo1 = 'MGLU_ER_3T20_POR.pdf'
arquivo2 = 'MGLU_ER_4T20_POR.pdf'

pdf_mesclado.append(arquivo1)
pdf_mesclado.append(arquivo2)

with Path(f'consolidadoAppend.pdf').open(mode='wb') as arquivo_final:
    pdf_mesclado.write(arquivo_final)

# Funcionalidades que podem ser úteis:

- Inserir arquivo no meio do outro
- Quero colocar dentro do Resultado do 4T20 os destaques do 3T20 para poder comparar os 2 dentro do mesmo relatório

In [69]:
pdf_mesclado = pyf.PdfMerger()

pdf_mesclado.append(arquivo1)
pdf_mesclado.merge(1, arquivo2)

with Path(f'consolidadoMerge.pdf').open(mode='wb') as arquivo_final:
    pdf_mesclado.write(arquivo_final)

- Rodar Página

In [70]:
pdf_rodar = pyf.PdfReader(arquivo1)

pdf_final = pyf.PdfWriter()
for pagina in pdf_rodar.pages:
    pagina.rotate(90)
    pdf_final.add_page(pagina)

with Path(f'Rodado.pdf').open(mode='wb') as arquivo_final:
    pdf_final.write(arquivo_final)

# Trabalhando com Textos e Informações Dentro do PDF

#### 1º Objetivo: Quero identificar como foram as Despesas com Vendas da MGLU
    - Pegar texto da página e identificar onde está essa informação

In [71]:
nome_arquivo = "MGLU_ER_3T20_POR.pdf"
arquivo = pyf.PdfReader(nome_arquivo)

num_paginas =   len(arquivo.pages) #arquivo.numPages
print(num_paginas)

informacoes = arquivo.metadata
print(informacoes)

24
{'/Title': 'DESEMPENHO FINANCEIRO CONSOLIDADO', '/Author': 'an_rezende', '/Subject': 'Receita Bruta', '/Creator': 'Microsoft® Office Word 2007', '/CreationDate': "D:20201109183121-03'00'", '/ModDate': "D:20201109183121-03'00'", '/Producer': 'Microsoft® Office Word 2007'}


In [72]:
texto_procurado = "| Despesas com Vendas"

i = 1
#percorrendo todas as paginas
for pagina in arquivo.pages:
    #pegar o que está escrito na página
    texto_pagina = pagina.extract_text()
    #verificar se dentro do texto da página, tem o teto procurado
    if texto_procurado in texto_pagina:
    #se tiver, me diz qual é o número da página    
        print(f'Está na página {i}')
        num_pagina = i
        texto_final = texto_pagina
    i+=1

Está na página 10


In [73]:
print(texto_final)

Divulgação de Resultados  
3T20 
10 
 
  
 
| Despesas Operacionais  
 
R$ milhões  3T20  
Ajustado   % RL  3T19  
Ajustado   % RL   Var(%)  9M20  
Ajustado   % RL  9M19  
Ajustado   % RL   Var(%)  
  Despesas com Vendas   (1.432,6)  -17,2%  (890,0)  -18,3%  61,0%  (3.487,2)  -18,2%  (2.309,1)  -17,1%  51,0%  
  Despesas Gerais e Administrativas   (240,7)  -2,9%  (207,1)  -4,3%  16,2%  (617,3)  -3,2%  (498,2)  -3,7%  23,9%  
 Subtotal      (1.673,3)  -20,1%      (1.097,1)  -22,6%  52,5%      (4.104,5)  -21,5%      (2.807,4)  -20,8%  46,2%  
  Perdas em Liquidação Duvidosa   (25,4)  -0,3%  (20,2)  -0,4%  25,4%  (84,5)  -0,4%  (45,8)  -0,3%  84,3%  
  Outras Receitas Operacionais, Líquidas   15,2  0,2%  15,3  0,3%  -0,6%  41,0  0,2%  44,0  0,3%  -6,8%  
  Total de Despesas Operacionais       (1.683,5)  -20,3%      (1.102,0)  -22,7%  52,8%      (4.148,0)  -21,7%      (2.809,2)  -20,8%  47,7%  
 
| Despesas com Vendas  
 
No 3T20, as despesas com vendas totalizaram R$1.432,6 milhões, equiv

In [74]:
texto_final = texto_final.replace("  ", " ")
texto_final = texto_final.replace("\n \n", "&&&&&&&")
texto_final = texto_final.replace("\n", "")
texto_final = texto_final.replace("&&&&&&&", "\n \n")
print(texto_final)

Divulgação de Resultados 3T20 10 
 
 
 
| Despesas Operacionais 
 
R$ milhões 3T20 Ajustado  % RL 3T19 Ajustado  % RL  Var(%) 9M20 Ajustado  % RL 9M19 Ajustado  % RL  Var(%)  Despesas com Vendas  (1.432,6) -17,2% (890,0) -18,3% 61,0% (3.487,2) -18,2% (2.309,1) -17,1% 51,0%  Despesas Gerais e Administrativas  (240,7) -2,9% (207,1) -4,3% 16,2% (617,3) -3,2% (498,2) -3,7% 23,9%  Subtotal   (1.673,3) -20,1%   (1.097,1) -22,6% 52,5%   (4.104,5) -21,5%   (2.807,4) -20,8% 46,2%  Perdas em Liquidação Duvidosa  (25,4) -0,3% (20,2) -0,4% 25,4% (84,5) -0,4% (45,8) -0,3% 84,3%  Outras Receitas Operacionais, Líquidas  15,2 0,2% 15,3 0,3% -0,6% 41,0 0,2% 44,0 0,3% -6,8%  Total de Despesas Operacionais    (1.683,5) -20,3%   (1.102,0) -22,7% 52,8%   (4.148,0) -21,7%   (2.809,2) -20,8% 47,7% 
 
| Despesas com Vendas 
 
No 3T20, as despesas com vendas totalizaram R$1.432,6 milhões, equivalentes a 17,2% da receita líquida, 1,1 p.p. menor que no 3T19 , principalmente devido ao forte crescimento das vendas

In [78]:
posicao = texto_final.find("| Despesas com Vendas")
posicao_final = texto_final.find("|", posicao + 1)
print(posicao)
print(posicao_final)
texto_despesa = texto_final[posicao:posicao_final]
print(texto_despesa)

789
1300
| Despesas com Vendas 
 
No 3T20, as despesas com vendas totalizaram R$1.432,6 milhões, equivalentes a 17,2% da receita líquida, 1,1 p.p. menor que no 3T19 , principalmente devido ao forte crescimento das vendas . Vale ressaltar que a Companhia conseguiu diluir as despesas com vendas m esmo investi ndo em maior nível de serviço, especialmente em atendimento e logística. 
 
Nos 9M20, as despesas com vendas totalizaram R$3.487,2 milhões, equivalentes a 18,2% da receita líquida (+1,1 p.p. versus os 9M19). 
 



#### 2º Objetivo: Quero analisar o DRE (sem ajuste - Página 5)
    - Para ler tabelas em pdf, use o tabula (é ninja)

In [92]:
import tabula

df = tabula.io.read_pdf("MGLU_ER_3T20_POR.pdf", pages=5)
tabela = df[0]
tabela = tabela.dropna(how='all')
tabela = tabela.dropna(how='all', axis=1)
tabela.columns = tabela.iloc[0]
tabela = tabela.drop(tabela.index[0])
tabela = tabela.reset_index(drop=True)
display(tabela)
tabela.to_excel("Arquivo.xlsx")

,R$ milhões (exceto quando indicado),3T2,3T1,Var(%,9M2,9M1,Var(%
0,Vendas Totais1 (incluindo marketplace),"12.355,5","6.817,6","81,2","28.584,","18.282,6","56,3"
1,Receita Bruta,"10.349,5","5.999,4","72,5","23.652,","16.508,8","43,3"
2,Receita Líquida,"8.308,3","4.864,2","70,8","19.111,","13.501,3","41,6"
3,Lucro Bruto,"2.178,7","1.424,9","52,9","5.034,","3.728,6","35,0"
4,Margem Bruta,"26,2","29,3","-3,1 p","26,3","27,6","-1,3 p"
5,EBITDA,"546,1","501,2","9,0","1.022,","1.276,5","-19,9"
6,Margem EBITDA,"6,6","10,3","-3,7 p","5,3","9,5","-4,2 p"
7,Lucro Líquido,"206,0","235,1","-12,4","172,","753,8","-77,2"
8,Margem Líquida,"2,5","4,8","-2,3 p","0,9","5,6","-4,7 p"
9,Lucro Bruto - Ajustado,"2.178,7","1.488,9","46,3","5.034,","3.964,6","27,0"


#### 3º Objetivo: Quero analisar o Capital de Giro e os Investimentos (ambas as tabelas na página 12)
    - Páginas com mais de 1 tabela

In [94]:
df = tabula.io.read_pdf("MGLU_ER_3T20_POR.pdf", pages=12)
for tabela in df:
    tabela = tabela.dropna(how='all')
    tabela = tabela.dropna(how='all', axis=1)
    display(tabela)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,(+) Contas a Receber (sem Cartões de Crédito),"(26,7)","706,3","680,8","781,3","794,0","733,0"
1,(+) Estoques,"2.120,2","5.005,9","4.198,2","4.075,5","3.801,8","2.885,7"
2,(+) Partes Relacionadas (sem Cartão Luiza),"(10,5)","71,3","80,4","77,1","100,6","81,8"
3,(+) Impostos a Recuperar,"186,3","932,0","748,9","877,4","864,1","745,7"
4,(+) Outros Ativos,"(56,6)","88,5","100,2","143,5","136,3","145,1"
5,(+) Ativos Circulantes Operacionais,"2.212,7","6.804,0","5.808,6","5.954,8","5.696,8","4.591,3"
7,(-) Fornecedores,"2.301,5","6.104,3","5.334,0","4.132,7","5.934,9","3.802,8"
8,(-) Repasses e outros depósitos,"627,3","627,3","639,3","235,9",-,-
9,"(-) Salários, Férias e Encargos Sociais","95,0","444,7","329,0","263,3","354,7","349,8"
10,(-) Impostos a Recolher,"90,7","299,6","206,4","176,9","352,0","208,8"


,R$ milhões,3T20,%,3T19,%.1,Var(%),9M20,%.2,9M20.1,%.3,Var(%).1
1,Lojas Novas,"21,2",14%,"94,6",51%,-78%,"69,0",21%,"121,9",31%,-43%
2,Reformas,"6,1",4%,"8,2",4%,-26%,"14,6",4%,"38,2",10%,-62%
3,Tecnologia,"69,1",45%,"32,6",18%,112%,"147,6",45%,"84,4",22%,75%
4,Logística,"36,3",24%,"32,8",18%,11%,"62,1",19%,"107,4",27%,-42%
5,Outros,"21,5",14%,"17,7",10%,22%,"35,7",11%,"38,8",10%,-8%
6,Total,"154,2",100%,"186,0",100%,-17%,"329,1",100%,"390,7",100%,-16%


#### O que fazer quando o tabula não consegue ler alguma linha da tabela? Como o cabeçalho, no nosso caso?

In [97]:
capital_giro = df[0]
capital_giro = capital_giro.dropna(how='all')
capital_giro = capital_giro.dropna(how='all', axis=1)
df2 = tabula.io.read_pdf("MGLU_ER_3T20_POR.pdf", pages=12, lattice = True)
titulo_capital_giro = df2[0]
titulo_capital_giro = titulo_capital_giro.dropna(how='all')
titulo_capital_giro = titulo_capital_giro.dropna(how='all', axis=1)
titulo_capital_giro = titulo_capital_giro.iloc[0]
titulo_capital_giro = titulo_capital_giro.dropna()
capital_giro.columns = titulo_capital_giro
display(capital_giro)

1,R$ milhões,Dif 12U,set-2,jun-2,mar-2,dez-1,set-1
0,(+) Contas a Receber (sem Cartões de Crédito),"(26,7)","706,3","680,8","781,3","794,0","733,0"
1,(+) Estoques,"2.120,2","5.005,9","4.198,2","4.075,5","3.801,8","2.885,7"
2,(+) Partes Relacionadas (sem Cartão Luiza),"(10,5)","71,3","80,4","77,1","100,6","81,8"
3,(+) Impostos a Recuperar,"186,3","932,0","748,9","877,4","864,1","745,7"
4,(+) Outros Ativos,"(56,6)","88,5","100,2","143,5","136,3","145,1"
5,(+) Ativos Circulantes Operacionais,"2.212,7","6.804,0","5.808,6","5.954,8","5.696,8","4.591,3"
7,(-) Fornecedores,"2.301,5","6.104,3","5.334,0","4.132,7","5.934,9","3.802,8"
8,(-) Repasses e outros depósitos,"627,3","627,3","639,3","235,9",-,-
9,"(-) Salários, Férias e Encargos Sociais","95,0","444,7","329,0","263,3","354,7","349,8"
10,(-) Impostos a Recolher,"90,7","299,6","206,4","176,9","352,0","208,8"


# Outro método que pode ser útil algum dia: Captar Imagem em um pdf
    - biblioteca pikepdf

In [ ]:
from pikepdf import Pdf, PdfImage

filename = "MGLU_ER_3T20_POR.pdf"
example = Pdf.open(filename)

n_arquivo = 0
for i, pagina in enumerate(example.pages):
    for nome, imagem in pagina.images.items():
        imagem_salvar = PdfImage(imagem)
        imagem_salvar.extract_to(fileprefix=f'imgs/Pagina_{i:02}_{n_arquivo}')
        n_arquivo += 1

# Substituir texto no pdf tipo contrato

- Não recomendo fazer diretamente pelo Python. Realmente do que vi a melhor opção me parece o Word fazer isso
- Caso precise automatizar, automatize o processo fazendo ele pelo Word
- Quem quiser MUITO fazer isso pelo Python, tem um link aqui que vai te ajudar de uma solução que achei que funciona. Tem seus bugs/cuidados especiais, mas funciona: https://pdf.co/samples/pdf-co-web-api-replace-text-from-pdf-python-replace-text-from-url